<a href="https://colab.research.google.com/github/ralavender/Effects-of-Accents-Bias-in-Voice-Assisted-AI/blob/main/English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("~/Desktop/data120/cv-corpus-19.0-delta-2024-09-13-en/cv-corpus-19.0-delta-2024-09-13/en/other.tsv", sep="\t")

print(df.shape)
df.head()

In [ ]:
english = df[df.accents.str.contains("English", case=False, na=False)]

print(english.shape)
english.head()

In [ ]:
english.accents.value_counts()

In [ ]:
mask = (english.accents == "United States English")

us_english = english[mask]

print(us_english.shape)
us_english.head()

(3384, 13)


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,191ee24977127424a5ac34a754b12170b36bd23863cbda...,common_voice_en_40865211.mp3,ee71e29300d6bf97715142b1ddfeb9a46f27cb0ff4f580...,"With this transition to the big time, the band...",NaN,0,0,NaN,NaN,United States English,NaN,en,NaN
1,191ee24977127424a5ac34a754b12170b36bd23863cbda...,common_voice_en_40865212.mp3,ee6a9d5da68a933995ee3fc7fe679e019df35295ce24a2...,Local brothels recruited extra staff to cope w...,NaN,0,0,NaN,NaN,United States English,NaN,en,NaN
2,191ee24977127424a5ac34a754b12170b36bd23863cbda...,common_voice_en_40865213.mp3,ee79622517bc43e5958b41951c455320ae6588f5142448...,"With Fox on lead vocals, the threesome did two...",NaN,0,0,NaN,NaN,United States English,NaN,en,NaN
3,191ee24977127424a5ac34a754b12170b36bd23863cbda...,common_voice_en_40865214.mp3,ee67248663e5c083e777259bc983427e3a36f168b16768...,Miramax requested cuts be made and Christopher...,NaN,0,0,NaN,NaN,United States English,NaN,en,NaN
4,191ee24977127424a5ac34a754b12170b36bd23863cbda...,common_voice_en_40865215.mp3,ee7187768850017c14addbcb48664c31a2ee162f3a089f...,The Key allows customers to buy Plusbus for th...,NaN,0,0,NaN,NaN,United States English,NaN,en,NaN


In [ ]:
us_english.gender

In [ ]:
## import whisper
import whisper

# Load the Whisper model
model = whisper.load_model("base")  # You can choose "tiny", "base", "small", "medium", or "large"

# Transcribe audio
result = model.transcribe("C:/Users/ral/Desktop/data120/cv-corpus-19.0-delta-2024-09-13-en/cv-corpus-19.0-delta-2024-09-13/en/clips/common_voice_en_40865213.mp3")  # Replace with your audio file path

# Print the transcription
print(result["text"])


 With Fox on lead vocals, the three-sympti two short tours in Europe.


In [ ]:
base_path = "C:/Users/ral/Desktop/data120/cv-corpus-19.0-delta-2024-09-13-en/cv-corpus-19.0-delta-2024-09-13/en/clips/"

us_english['full_path'] = base_path + us_english['path']

file_list = us_english['full_path'].tolist()

In [ ]:
import pandas as pd
import whisper
from jiwer import wer, cer
import json

# Load Whisper model
model = whisper.load_model("base")

# Initialize results list
results = []

# Process each file in the DataFrame
for index, row in us_english.iterrows():
    file_path = row['full_path']
    ground_truth = row['sentence']

    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping.")
        continue

    print(f"Processing {file_path}...")

    # Transcribe the audio file
    transcription = model.transcribe(file_path)["text"]

    # Calculate WER and CER
    current_wer = wer(ground_truth, transcription)
    current_cer = cer(ground_truth, transcription)

    # Append results
    results.append({
        "file": file_path,
        "transcription": transcription,
        "ground_truth": ground_truth,
        "wer": current_wer,
        "cer": current_cer,
    })

# Save results to a JSON file
with open("subset_results.json", "w") as f:
    json.dump(results, f, indent=4)

print("Processing complete. Results saved to 'subset_results.json'.")


In [ ]:
len(results)

3384

In [ ]:
# Load results
with open("subset_results.json", "r") as f:
    results = json.load(f)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Calculate average WER and CER
average_wer = results_df['wer'].mean()
average_cer = results_df['cer'].mean()

print(f"Average WER: {average_wer:.2%}")
print(f"Average CER: {average_cer:.2%}")


Average WER: 19.32%
Average CER: 7.73%


In [ ]:
f_index = "C:/Users/ral/Desktop/data120/cv-corpus-19.0-delta-2024-09-13-en/cv-corpus-19.0-delta-2024-09-13/en/clips/" + us_english[us_english.gender == "female_feminine"].path
m_index = "C:/Users/ral/Desktop/data120/cv-corpus-19.0-delta-2024-09-13-en/cv-corpus-19.0-delta-2024-09-13/en/clips/" + us_english[us_english.gender == "male_masculine"].path

results_female = results_df[results_df['file'].isin(f_index)]
results_male = results_df[results_df['file'].isin(m_index)]

mean_wer = results_female['wer'].mean()
mean_cer = results_female['cer'].mean()

print(f"Mean WER for female speakers: {mean_wer:.2%}")
print(f"Mean CER for female speakers: {mean_cer:.2%}")

mean_wer = results_male['wer'].mean()
mean_cer = results_male['cer'].mean()

print(f"\nMean WER for male speakers: {mean_wer:.2%}")
print(f"Mean CER for male speakers: {mean_cer:.2%}")

Mean WER for female speakers: 20.65%
Mean CER for female speakers: 8.17%

Mean WER for male speakers: 13.38%
Mean CER for male speakers: 3.82%
